<a href="https://colab.research.google.com/github/aaronmj7/Assignment-3-LLM/blob/main/Assignment_3_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>